In [ ]:
#setup the environment
import os
import json
from openai import AzureOpenAI
from azure.core.exceptions import AzureError
import sys
import requests

apiKey = input("Enter your Azure OpenAI API Key: ")
apiEndpoint = input("Enter your Azure OpenAI API Endpoint: ")
model = input("Enter your model deployment name: ")
useFunction = True

client = AzureOpenAI(
    api_key = apiKey,  
    api_version = "2023-12-01-preview",
    azure_endpoint = apiEndpoint
)



In [117]:
def get_current_weather(obj={}):
    location = obj.get('location')
    try:
        unit = obj.get('unit')
    except:
        unit = "Fahrenheit"
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [118]:

def get_lat_long_weather(obj={}):
    latitude = str(obj.get('latitude'))
    longitude = str(obj.get('longitude'))
    response = requests.get(f'https://api.weather.gov/points/{latitude},{longitude}')
    #response = requests.get('https://api.weather.gov/points/38.8304,-77.1964')
    unit="fahrenheit"
    if response.status_code == 200:
         # Parse the response as JSON
        data = response.json()
        #print(data)
        office = data.get("properties",{}).get("gridId")
        gridx = data.get("properties",{}).get("gridX")
        gridy = data.get("properties",{}).get("gridY")

        print(office)
        print(gridx)
        print(gridy)
        # Make a request to the /gridpoints/{office}/{gridX},{gridY}/forecast endpoint
        response = requests.get(f'https://api.weather.gov/gridpoints/{office}/{gridx},{gridy}/forecast')
        if response.status_code == 200:
            # Parse the response as JSON
            data = response.json()
            #print(data)
            # Get the current temperature
            temperature = data.get("properties",{}).get("periods", [])[0].get("temperature")
            detailedForecast = data.get("properties",{}).get("periods", [])[0].get("detailedForecast")
            name = data.get("properties",{}).get("periods", [])[0].get("name")
            #print(temperature)
            # Return the temperature
            return json.dumps({"latitude": latitude, "longitude": longitude, 
                               "temperature": temperature, "unit": unit, 
                               "detailedForecast": detailedForecast,"name": name })
        else:
            print(f"Request failed with status code step 2 {response.status_code}")

    else:
        print(f"Request failed with status code step 1 {response.status_code}")


print("### get_lat_long_weather ###")
print( get_lat_long_weather( {'latitude':38.8304, 'longitude':-77.1964} ))

### get_lat_long_weather ###
LWX
91
68
{"latitude": "38.8304", "longitude": "-77.1964", "temperature": 55, "unit": "fahrenheit", "detailedForecast": "Sunny, with a high near 55. Northwest wind 22 to 25 mph, with gusts as high as 47 mph.", "name": "Today"}


In [119]:

    
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for only Tokyo, San Francisco, and Paris.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_lat_long_weather",
            "description": "Get the current weather All other locations given a latitude and longitude in fahrenheit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "The latitude of the location",
                    },
                    "longitude": {
                        "type": "number",
                        "description": "The longitude of the location",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        },
    }
]

available_functions = {
                "get_current_weather": get_current_weather,
                "get_lat_long_weather": get_lat_long_weather,
                } 


#System Prompt
messages = [{"role": "system", "content": "Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure."}]
print("Azure OpenAI client created and system prompt loaded.")


print("Function and tools defined")

Azure OpenAI client created and system prompt loaded.
Function and tools defined


In [120]:
#Example Questions
#Who are you?
#What is the temperature in paris?
#What is the temperature in Washington DC?
message = input("Your question: ")
messages.append({"role": "user", "content": message})

print(messages)

[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'What is the temperature in Annandale VA?'}]


In [125]:
print("Starting Messages:")
print(messages)
response = client.chat.completions.create(
                model=model, 
                messages=messages,
                tools=tools,
                tool_choice="auto",
            )

print("\n\n###########Full Response###########")
print(response)
print("\n\n###########Response to User###########")
print(response.choices[0].message.content)
print("\ntools:")
print(response.choices[0].message.tool_calls)
print("##############################")
#print("\n\n###########Filter###########")
#print(json.dumps(response.choices[0].content_filter_results, indent=4))
#print("############################")


if (response.choices[0].message.tool_calls):
    messages.append({"content": response.choices[0].message.content,
                "role": response.choices[0].message.role,
                "tool_calls":response.choices[0].message.tool_calls
                })
else:
        messages.append({"content": response.choices[0].message.content,
                "role": response.choices[0].message.role })

print("\n\n###########Ending Messages###########")
print(messages)

Starting Messages:
[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'What is the temperature in Annandale VA?'}, {'content': None, 'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_azqSw4SuY3FXyxOZGBzPqWCi', function=Function(arguments='{"latitude":38.8304,"longitude":-77.1964}', name='get_lat_long_weather'), type='function')]}, {'tool_call_id': 'call_azqSw4SuY3FXyxOZGBzPqWCi', 'role': 'tool', 'name': 'get_lat_long_weather', 'content': '{"latitude": "38.8304", "longitude": "-77.1964", "temperature": 55, "unit": "fahrenheit", "detailedForecast": "Sunny, with a high near 55. Northwest wind 22 to 25 mph, with gusts as high as 47 mph.", "name": "Today"}'}]


###########Full Response###########
ChatCompletion(id='chatcmpl-91cCyRxDhJKBTxnliPwnUVbzNVV1U', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMe

In [122]:
tool_calls = response.choices[0].message.tool_calls

print("###########Tool Calls###########")
print(tool_calls)

###########Tool Calls###########
[ChatCompletionMessageToolCall(id='call_azqSw4SuY3FXyxOZGBzPqWCi', function=Function(arguments='{"latitude":38.8304,"longitude":-77.1964}', name='get_lat_long_weather'), type='function')]


In [123]:
# Select the tool to use
toolItr = 0
#toolItr = input("Enter the tool index to use: ")

tool_call = tool_calls[toolItr]


# Call the function
function_name  = tool_call.function.name
function_to_call = available_functions[function_name]
function_args  = json.loads(tool_call.function.arguments)
function_response = function_to_call(obj=function_args)

print("\n\n###########Function Response###########")
print(json.dumps(function_response, indent=4))



LWX
91
68


###########Function Response###########
"{\"latitude\": \"38.8304\", \"longitude\": \"-77.1964\", \"temperature\": 55, \"unit\": \"fahrenheit\", \"detailedForecast\": \"Sunny, with a high near 55. Northwest wind 22 to 25 mph, with gusts as high as 47 mph.\", \"name\": \"Today\"}"


In [124]:
#Append the function response to the messages

messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

print("\n\n###########Ending Messages###########")
print(messages)



###########Ending Messages###########
[{'role': 'system', 'content': 'Your name is Azure Andy and you are an assistant that is a large language model trained by OpenAI running on Azure.'}, {'role': 'user', 'content': 'What is the temperature in Annandale VA?'}, {'content': None, 'role': 'assistant', 'tool_calls': [ChatCompletionMessageToolCall(id='call_azqSw4SuY3FXyxOZGBzPqWCi', function=Function(arguments='{"latitude":38.8304,"longitude":-77.1964}', name='get_lat_long_weather'), type='function')]}, {'tool_call_id': 'call_azqSw4SuY3FXyxOZGBzPqWCi', 'role': 'tool', 'name': 'get_lat_long_weather', 'content': '{"latitude": "38.8304", "longitude": "-77.1964", "temperature": 55, "unit": "fahrenheit", "detailedForecast": "Sunny, with a high near 55. Northwest wind 22 to 25 mph, with gusts as high as 47 mph.", "name": "Today"}'}]
